In [1]:
import configparser
import psycopg2
import pandas as pd

In [32]:
import pandas.io.sql as sqlio
from IPython.display import display

In [6]:
%load_ext sql

In [3]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [4]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [5]:
conn.autocommit=True

**Get top 10 artist**

In [33]:
top_artists_query = f"""
SELECT artist_dim.name AS artist, COUNT(*) AS count 
FROM songplay_fact JOIN artist_dim ON songplay_fact.artist_id = artist_dim.artist_id
GROUP BY artist_dim.name
ORDER BY count DESC
LIMIT 10
"""
result = sqlio.read_sql_query(top_artists_query, conn)
display(result)

,artist,count
0,Dwight Yoakam,37
1,Kid Cudi / Kanye West / Common,10
2,Kid Cudi,10
3,Ron Carter,9
4,Lonnie Gordon,9
5,B.o.B,8
6,Usher,6
7,Usher featuring Jermaine Dupri,6
8,Muse,6
9,Richard Hawley And Death Ramps_ Arctic Monkeys,5


**Get top 10 songs**

In [35]:
top_songs_query = f"""
SELECT song_dim.title AS song_name, COUNT(*) AS count 
FROM songplay_fact JOIN song_dim ON songplay_fact.song_id = song_dim.song_id
GROUP BY song_dim.title
ORDER BY count DESC
LIMIT 10
"""
result = sqlio.read_sql_query(top_songs_query, conn)
result

,song_name,count
0,You're The One,37
1,I CAN'T GET STARTED,9
2,Catch You Baby (Steve Pitron & Max Sanna Radio...,9
3,Nothin' On You [feat. Bruno Mars] (Album Version),8
4,Hey Daddy (Daddy's Home),6
5,Make Her Say,5
6,Up Up & Away,5
7,Unwell (Album Version),4
8,Mr. Jones,4
9,Supermassive Black Hole (Album Version),4


**What time do people listen to songs most**

In [36]:
top_hour_query = f"""
SELECT time_dim.hour AS hour_of_the_day, COUNT(*) AS count 
FROM songplay_fact JOIN time_dim ON songplay_fact.start_time = time_dim.start_time
GROUP BY time_dim.hour
ORDER BY count DESC
LIMIT 10
"""
result = sqlio.read_sql_query(top_hour_query, conn)
result

,hour_of_the_day,count
0,17,40
1,15,26
2,18,26
3,16,24
4,20,18
5,8,18
6,19,16
7,14,16
8,11,16
9,13,14
